# 0. Import Data 

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

# Import data from Amazon S3 Bucket
role = get_execution_role()
bucket='project-b-data'
data_key = 'proj-b/Tape_with_LOB/Tape_with_LOB1.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

data = pd.read_csv(data_location)
data.head()

/home/ec2-user/anaconda3/envs/mxnet_latest_p37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-01-04,8.192,203,2.0,S26,B24,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
1,2022-01-04,8.192,199,3.0,S26,B08,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,..."
2,2022-01-04,10.336,196,3.0,S06,B07,"[[196, 3], [195, 4], [187, 8], [182, 1], [156,...","[[197, 4], [198, 3], [201, 1], [202, 4], [209,..."
3,2022-01-04,10.672,196,4.0,S01,B01,"[[195, 4], [187, 5], [183, 3], [182, 1], [156,...","[[196, 4], [201, 1], [203, 4], [209, 9], [214,..."
4,2022-01-04,11.040,201,1.0,S12,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,..."


In [3]:
# Import B02 data from Amazon S3 Bucket (validation set)
role2 = get_execution_role()
bucket2='project-b-new'
data_key2 = 'Tape_with_LOB/Tape_with_LOB_B02.csv'
data_location2 = 's3://{}/{}'.format(bucket2, data_key2)

data2 = pd.read_csv(data_location2)
data2.head()

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-01-04,10.778,253,1.0,S01,B08,"[[245, 5], [243, 1], [239, 1], [238, 1], [237,...","[[253, 1], [254, 5], [256, 6], [257, 1], [258,..."
1,2022-01-04,12.631,254,1.0,S03,B22,"[[253, 5], [252, 1], [250, 1], [238, 5], [231,...","[[254, 5], [256, 6], [257, 1], [258, 3], [259,..."
2,2022-01-04,12.835,254,4.0,S03,B12,"[[253, 5], [252, 1], [238, 5], [231, 2], [221,...","[[254, 4], [256, 6], [257, 1], [258, 3], [259,..."
3,2022-01-04,14.144,256,1.0,S10,B12,"[[255, 3], [254, 1], [252, 1], [238, 5], [235,...","[[256, 6], [257, 1], [258, 5], [259, 6], [260,..."
4,2022-01-04,14.484,256,1.0,S10,B28,"[[255, 3], [252, 3], [241, 1], [238, 4], [235,...","[[256, 5], [257, 1], [258, 3], [259, 6], [260,..."


In [4]:
# Import B03 data from Amazon S3 Bucket (test set)
role3 = get_execution_role()
bucket3='project-b-new'
data_key3 = 'Tape_with_LOB/Tape_with_LOB_B03.csv'
data_location3 = 's3://{}/{}'.format(bucket3, data_key3)

data3 = pd.read_csv(data_location3)
data3.head()

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask
0,2022-01-04,7.089,263,3.0,S19,B15,"[[253, 4], [251, 2], [250, 2], [249, 4], [247,...","[[263, 8], [264, 2], [265, 5], [266, 3], [289,..."
1,2022-01-04,7.089,263,2.0,S13,B15,"[[253, 4], [251, 2], [250, 2], [249, 4], [247,...","[[263, 8], [264, 2], [265, 5], [266, 3], [289,..."
2,2022-01-04,8.619,259,1.0,S10,B02,"[[258, 4], [257, 1], [256, 2], [255, 2], [250,...","[[259, 3], [261, 5], [263, 3], [264, 4], [302,..."
3,2022-01-04,8.908,259,2.0,S10,B18,"[[258, 4], [256, 2], [255, 2], [249, 4], [246,...","[[259, 2], [261, 5], [263, 3], [264, 4], [302,..."
4,2022-01-04,9.588,260,2.0,S17,B11,"[[259, 4], [255, 2], [249, 4], [246, 4], [241,...","[[260, 2], [261, 5], [263, 5], [293, 2], [314,..."


---
# 1. Data Preparation 

<b>1.1 Adding features to the LOB data for reinforcement learning model. New features include:
1. Highest bid price
2. Quantity of stocks with highest bid price
3. Lowest ask price
4. Quantity of stocks with lowest ask price
5. Bid-ask spread
6. Mid-price
7. Market depth

In [5]:
import ast 
import numpy as np

In [6]:
def get_highest_bid_price(bid):
    if type(bid) != float:
        new_bid = ast.literal_eval(bid)
        new_bid = np.array(new_bid)
        if new_bid.size != 0:
            return new_bid[:,0][0]
        else:
            return 0
    else:
        return 0

In [7]:
data['highest_bid_price'] = data.apply(lambda bid: get_highest_bid_price(bid['Bid']), axis=1)

In [8]:
def get_highest_bid_quantity(bid):
    if type(bid) != float:
        new_bid = ast.literal_eval(bid)
        new_bid = np.array(new_bid)
        if new_bid.size != 0:
            return max(new_bid[:,1])
        else:
            return 0
    else:
        return 0

In [9]:
data['highest_bid_quantity'] = data.apply(lambda bid: get_highest_bid_quantity(bid['Bid']), axis=1)

In [10]:
def get_lowest_ask_price(ask):
    if type(ask) != float:
        new_ask = ast.literal_eval(ask)
        new_ask = np.array(new_ask)
        if new_ask.size != 0:
            return new_ask[:,0][0]
        else:
            return 0
    else:
        return 0

In [11]:
data['lowest_ask_price'] = data.apply(lambda ask: get_lowest_ask_price(ask['Ask']), axis=1)

In [12]:
def get_lowest_ask_quantity(ask):
    if type(ask) != float:
        new_ask = ast.literal_eval(ask)
        new_ask = np.array(new_ask)
        if new_ask.size != 0:
            return min(new_ask[:,1])
        else:
            return 0
    else:
        return 0

In [13]:
data['lowest_ask_quantity'] = data.apply(lambda ask: get_lowest_ask_quantity(ask['Ask']), axis=1)
data['Spread'] = data['lowest_ask_price']-data['highest_bid_price']
data['Mid_price'] =(data['lowest_ask_price']+data['highest_bid_price'])/2

In [14]:
def get_Market_depth(ask,bid):
    if type(ask) != float and type(bid) !=float:
        new_ask = ast.literal_eval(ask)
        new_ask = np.array(new_ask)
        new_bid = ast.literal_eval(bid)
        new_bid = np.array(new_bid)
        if new_bid.size != 0:
            return new_ask[:,0][-1]-new_bid[:,0][-1]
        else:
            return 0
    else:
        return 0

In [15]:
data['Market_depth']=list(map(lambda ask,bid:get_Market_depth(ask,bid), data['Ask'], data['Bid']))

In [16]:
data.head()

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask,highest_bid_price,highest_bid_quantity,lowest_ask_price,lowest_ask_quantity,Spread,Mid_price,Market_depth
0,2022-01-04,8.192,203,2.0,S26,B24,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,...",203,5,207,2,4,205.0,747
1,2022-01-04,8.192,199,3.0,S26,B08,"[[203, 2], [199, 3], [195, 4], [187, 5], [181,...","[[207, 4], [211, 4], [214, 2], [220, 5], [226,...",203,5,207,2,4,205.0,747
2,2022-01-04,10.336,196,3.0,S06,B07,"[[196, 3], [195, 4], [187, 8], [182, 1], [156,...","[[197, 4], [198, 3], [201, 1], [202, 4], [209,...",196,8,197,1,1,196.5,764
3,2022-01-04,10.672,196,4.0,S01,B01,"[[195, 4], [187, 5], [183, 3], [182, 1], [156,...","[[196, 4], [201, 1], [203, 4], [209, 9], [214,...",195,5,196,1,1,195.5,764
4,2022-01-04,11.040,201,1.0,S12,B31,"[[192, 4], [187, 5], [182, 1], [176, 5], [168,...","[[201, 1], [203, 4], [205, 5], [209, 4], [214,...",192,5,201,1,9,196.5,592


In [17]:
# Validation set B02
data2['highest_bid_price'] = data2.apply(lambda bid: get_highest_bid_price(bid['Bid']), axis=1)
data2['highest_bid_quantity'] = data2.apply(lambda bid: get_highest_bid_quantity(bid['Bid']), axis=1)
data2['lowest_ask_price'] = data2.apply(lambda ask: get_lowest_ask_price(ask['Ask']), axis=1)
data2['lowest_ask_quantity'] = data2.apply(lambda ask: get_lowest_ask_quantity(ask['Ask']), axis=1)
data2['Spread'] = data2['lowest_ask_price']-data2['highest_bid_price']
data2['Mid_price'] =(data2['lowest_ask_price']+data2['highest_bid_price'])/2
data2['Market_depth']=list(map(lambda ask,bid:get_Market_depth(ask,bid), data2['Ask'], data2['Bid']))
data2.head()

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask,highest_bid_price,highest_bid_quantity,lowest_ask_price,lowest_ask_quantity,Spread,Mid_price,Market_depth
0,2022-01-04,10.778,253,1.0,S01,B08,"[[245, 5], [243, 1], [239, 1], [238, 1], [237,...","[[253, 1], [254, 5], [256, 6], [257, 1], [258,...",245,5,253,1,8,249.0,713
1,2022-01-04,12.631,254,1.0,S03,B22,"[[253, 5], [252, 1], [250, 1], [238, 5], [231,...","[[254, 5], [256, 6], [257, 1], [258, 3], [259,...",253,8,254,1,1,253.5,630
2,2022-01-04,12.835,254,4.0,S03,B12,"[[253, 5], [252, 1], [238, 5], [231, 2], [221,...","[[254, 4], [256, 6], [257, 1], [258, 3], [259,...",253,8,254,1,1,253.5,732
3,2022-01-04,14.144,256,1.0,S10,B12,"[[255, 3], [254, 1], [252, 1], [238, 5], [235,...","[[256, 6], [257, 1], [258, 5], [259, 6], [260,...",255,5,256,1,1,255.5,700
4,2022-01-04,14.484,256,1.0,S10,B28,"[[255, 3], [252, 3], [241, 1], [238, 4], [235,...","[[256, 5], [257, 1], [258, 3], [259, 6], [260,...",255,4,256,1,1,255.5,700


In [18]:
# Test set B03
data3['highest_bid_price'] = data3.apply(lambda bid: get_highest_bid_price(bid['Bid']), axis=1)
data3['highest_bid_quantity'] = data3.apply(lambda bid: get_highest_bid_quantity(bid['Bid']), axis=1)
data3['lowest_ask_price'] = data3.apply(lambda ask: get_lowest_ask_price(ask['Ask']), axis=1)
data3['lowest_ask_quantity'] = data3.apply(lambda ask: get_lowest_ask_quantity(ask['Ask']), axis=1)
data3['Spread'] = data3['lowest_ask_price']-data3['highest_bid_price']
data3['Mid_price'] =(data3['lowest_ask_price']+data3['highest_bid_price'])/2
data3['Market_depth']=list(map(lambda ask,bid:get_Market_depth(ask,bid), data3['Ask'], data3['Bid']))
data3.head()

,Date,Time,Price,Quantity,Party1,Party2,Bid,Ask,highest_bid_price,highest_bid_quantity,lowest_ask_price,lowest_ask_quantity,Spread,Mid_price,Market_depth
0,2022-01-04,7.089,263,3.0,S19,B15,"[[253, 4], [251, 2], [250, 2], [249, 4], [247,...","[[263, 8], [264, 2], [265, 5], [266, 3], [289,...",253,5,263,2,10,258.0,302
1,2022-01-04,7.089,263,2.0,S13,B15,"[[253, 4], [251, 2], [250, 2], [249, 4], [247,...","[[263, 8], [264, 2], [265, 5], [266, 3], [289,...",253,5,263,2,10,258.0,302
2,2022-01-04,8.619,259,1.0,S10,B02,"[[258, 4], [257, 1], [256, 2], [255, 2], [250,...","[[259, 3], [261, 5], [263, 3], [264, 4], [302,...",258,5,259,2,1,258.5,766
3,2022-01-04,8.908,259,2.0,S10,B18,"[[258, 4], [256, 2], [255, 2], [249, 4], [246,...","[[259, 2], [261, 5], [263, 3], [264, 4], [302,...",258,6,259,2,1,258.5,631
4,2022-01-04,9.588,260,2.0,S17,B11,"[[259, 4], [255, 2], [249, 4], [246, 4], [241,...","[[260, 2], [261, 5], [263, 5], [293, 2], [314,...",259,5,260,2,1,259.5,308


<b>1.2 Adding features using transaction (tape) data for reinforcement learning model. New features include:
1. Highest price in a given time period 
2. Lowest price in a given time period
3. Opening prices of a trading session
4. Closing prices of a trading session
5. Volume of transactions in a given time period

In [19]:
#Drop columns that are not needed
df = data.drop(columns=['Party1','Party2','Bid','Ask'])
df.head()

,Date,Time,Price,Quantity,highest_bid_price,highest_bid_quantity,lowest_ask_price,lowest_ask_quantity,Spread,Mid_price,Market_depth
0,2022-01-04,8.192,203,2.0,203,5,207,2,4,205.0,747
1,2022-01-04,8.192,199,3.0,203,5,207,2,4,205.0,747
2,2022-01-04,10.336,196,3.0,196,8,197,1,1,196.5,764
3,2022-01-04,10.672,196,4.0,195,5,196,1,1,195.5,764
4,2022-01-04,11.040,201,1.0,192,5,201,1,9,196.5,592


In [20]:
# new data frame with split value columns
new = df["Date"].str.split("-", n = 2, expand = True)
  
# making separate first name column from new data frame
df['Y'] = new[0]
# making separate last name column from new data frame
df['M'] = new[1]
df['D'] = new[2]

df[["Y", "M","D"]] = df[["Y", "M","D"]].apply(pd.to_numeric)

In [21]:
import datetime

#Create datetime column based on  given Time column
from datetime import datetime, timedelta
from datetime import datetime, datetime

dt = []
for i in range(len(df)):
    start = datetime(df['Y'][i], df['M'][i], df['D'][i], 8, 0, 0) #start time when market opens
    dt.append(start + timedelta(seconds = df["Time"][i]))

df['DateTime'] = dt
df.head()

,Date,Time,Price,Quantity,highest_bid_price,highest_bid_quantity,lowest_ask_price,lowest_ask_quantity,Spread,Mid_price,Market_depth,Y,M,D,DateTime
0,2022-01-04,8.192,203,2.0,203,5,207,2,4,205.0,747,2022,1,4,2022-01-04 08:00:08.192
1,2022-01-04,8.192,199,3.0,203,5,207,2,4,205.0,747,2022,1,4,2022-01-04 08:00:08.192
2,2022-01-04,10.336,196,3.0,196,8,197,1,1,196.5,764,2022,1,4,2022-01-04 08:00:10.336
3,2022-01-04,10.672,196,4.0,195,5,196,1,1,195.5,764,2022,1,4,2022-01-04 08:00:10.672
4,2022-01-04,11.040,201,1.0,192,5,201,1,9,196.5,592,2022,1,4,2022-01-04 08:00:11.040


In [22]:
#High, low, open and close prices for every 5 minutes interval
prices = df.resample('1H', on='DateTime').Price.agg(['min','max', 'first','last'])
prices.reset_index(inplace=True)
prices.head()

,DateTime,min,max,first,last
0,2022-01-04 08:00:00,179.0,218.0,203.0,207.0
1,2022-01-04 09:00:00,186.0,221.0,217.0,189.0
2,2022-01-04 10:00:00,189.0,228.0,189.0,227.0
3,2022-01-04 11:00:00,194.0,228.0,227.0,225.0
4,2022-01-04 12:00:00,195.0,231.0,225.0,224.0


In [23]:
df_Q = df.drop(columns=['Date', 'Time', 'Price', 'Y', 'M', 'D','highest_bid_price','highest_bid_quantity','lowest_ask_price','lowest_ask_quantity','Spread','Mid_price','Market_depth'])
df_Q.head()

,Quantity,DateTime
0,2.0,2022-01-04 08:00:08.192
1,3.0,2022-01-04 08:00:08.192
2,3.0,2022-01-04 08:00:10.336
3,4.0,2022-01-04 08:00:10.672
4,1.0,2022-01-04 08:00:11.040


In [24]:
df_Q[["Quantity"]] = df[["Quantity"]].apply(pd.to_numeric)

qty = df_Q.resample('1H', on='DateTime').sum()
qty

,Quantity
DateTime,
2022-01-04 08:00:00,13908.0
2022-01-04 09:00:00,14092.0
2022-01-04 10:00:00,13830.0
2022-01-04 11:00:00,13692.0
2022-01-04 12:00:00,13911.0
...,...
2022-04-29 12:00:00,13975.0
2022-04-29 13:00:00,14139.0
2022-04-29 14:00:00,13812.0


In [25]:
Quantity = qty['Quantity'].tolist()

prices['Quantity'] = Quantity
prices.dropna(subset=['min'], inplace=True)
prices.head()

,DateTime,min,max,first,last,Quantity
0,2022-01-04 08:00:00,179.0,218.0,203.0,207.0,13908.0
1,2022-01-04 09:00:00,186.0,221.0,217.0,189.0,14092.0
2,2022-01-04 10:00:00,189.0,228.0,189.0,227.0,13830.0
3,2022-01-04 11:00:00,194.0,228.0,227.0,225.0,13692.0
4,2022-01-04 12:00:00,195.0,231.0,225.0,224.0,13911.0


In [26]:
prices.rename(columns = {'DateTime':'Date', 'min':'Low', 'max':'High', 'first':'Open', 'last':'Close', 'Quantity':'Volume'}, inplace = True)
prices.set_index('Date', inplace=True)
prices

,Low,High,Open,Close,Volume
Date,,,,,
2022-01-04 08:00:00,179.0,218.0,203.0,207.0,13908.0
2022-01-04 09:00:00,186.0,221.0,217.0,189.0,14092.0
2022-01-04 10:00:00,189.0,228.0,189.0,227.0,13830.0
2022-01-04 11:00:00,194.0,228.0,227.0,225.0,13692.0
2022-01-04 12:00:00,195.0,231.0,225.0,224.0,13911.0
...,...,...,...,...,...
2022-04-29 12:00:00,105.0,136.0,126.0,135.0,13975.0
2022-04-29 13:00:00,105.0,136.0,135.0,131.0,14139.0
2022-04-29 14:00:00,104.0,135.0,131.0,135.0,13812.0


In [27]:
df_Lob = df.drop(columns=['Quantity','Date', 'Time', 'Price', 'Y', 'M', 'D'])
df_Lob.head()

,highest_bid_price,highest_bid_quantity,lowest_ask_price,lowest_ask_quantity,Spread,Mid_price,Market_depth,DateTime
0,203,5,207,2,4,205.0,747,2022-01-04 08:00:08.192
1,203,5,207,2,4,205.0,747,2022-01-04 08:00:08.192
2,196,8,197,1,1,196.5,764,2022-01-04 08:00:10.336
3,195,5,196,1,1,195.5,764,2022-01-04 08:00:10.672
4,192,5,201,1,9,196.5,592,2022-01-04 08:00:11.040


In [28]:
lob = df_Lob.resample('1H', on='DateTime').last()
lob.head()

,highest_bid_price,highest_bid_quantity,lowest_ask_price,lowest_ask_quantity,Spread,Mid_price,Market_depth,DateTime
DateTime,,,,,,,,
2022-01-04 08:00:00,207.0,5.0,237.0,1.0,30.0,222.0,670.0,2022-01-04 08:59:58.272
2022-01-04 09:00:00,173.0,4.0,189.0,1.0,16.0,181.0,598.0,2022-01-04 09:59:59.984
2022-01-04 10:00:00,227.0,11.0,241.0,1.0,14.0,234.0,731.0,2022-01-04 10:59:59.904
2022-01-04 11:00:00,225.0,14.0,400.0,1.0,175.0,312.5,685.0,2022-01-04 11:59:58.848
2022-01-04 12:00:00,224.0,5.0,315.0,1.0,91.0,269.5,615.0,2022-01-04 12:59:58.720


In [29]:
lob.dropna(subset=['highest_bid_price'], inplace=True)

In [30]:
dataframe = pd.concat([prices,lob],axis=1)
dataframe = dataframe.drop(columns=['DateTime'])

In [31]:
dataframe.index.name = 'DateTime'
dataframe.head()

,Low,High,Open,Close,Volume,highest_bid_price,highest_bid_quantity,lowest_ask_price,lowest_ask_quantity,Spread,Mid_price,Market_depth
DateTime,,,,,,,,,,,,
2022-01-04 08:00:00,179.0,218.0,203.0,207.0,13908.0,207.0,5.0,237.0,1.0,30.0,222.0,670.0
2022-01-04 09:00:00,186.0,221.0,217.0,189.0,14092.0,173.0,4.0,189.0,1.0,16.0,181.0,598.0
2022-01-04 10:00:00,189.0,228.0,189.0,227.0,13830.0,227.0,11.0,241.0,1.0,14.0,234.0,731.0
2022-01-04 11:00:00,194.0,228.0,227.0,225.0,13692.0,225.0,14.0,400.0,1.0,175.0,312.5,685.0
2022-01-04 12:00:00,195.0,231.0,225.0,224.0,13911.0,224.0,5.0,315.0,1.0,91.0,269.5,615.0


In [32]:
# Validation set B02
#Drop columns that are not needed
df2 = data2.drop(columns=['Party1','Party2','Bid','Ask'])

# new data frame with split value columns
new2 = df2["Date"].str.split("-", n = 2, expand = True)
  
# making separate first name column from new data frame
df2['Y'] = new2[0]
# making separate last name column from new data frame
df2['M'] = new2[1]
df2['D'] = new2[2]

df2[["Y", "M","D"]] = df2[["Y", "M","D"]].apply(pd.to_numeric)

#Create datetime column based on  given Time column
dt2 = []
for i in range(len(df2)):
    start2 = datetime(df2['Y'][i], df2['M'][i], df2['D'][i], 8, 0, 0) #start time when market opens
    dt2.append(start2 + timedelta(seconds = df2["Time"][i]))

df2['DateTime'] = dt2

#High, low, open and close prices for every 1 day interval
prices2 = df2.resample('1D', on='DateTime').Price.agg(['min','max', 'first','last'])
prices2.reset_index(inplace=True)

df_Q2 = df2.drop(columns=['Date', 'Time', 'Price', 'Y', 'M', 'D','highest_bid_price','highest_bid_quantity','lowest_ask_price','lowest_ask_quantity','Spread','Mid_price','Market_depth'])

df_Q2[["Quantity"]] = df2[["Quantity"]].apply(pd.to_numeric)

qty2 = df_Q2.resample('1D', on='DateTime').sum()

Quantity2 = qty2['Quantity'].tolist()

prices2['Quantity'] = Quantity2
prices2.dropna(subset=['min'], inplace=True)

prices2.rename(columns = {'DateTime':'Date', 'min':'Low', 'max':'High', 'first':'Open', 'last':'Close', 'Quantity':'Volume'}, inplace = True)
prices2.set_index('Date', inplace=True)

df_Lob2 = df2.drop(columns=['Quantity','Date', 'Time', 'Price', 'Y', 'M', 'D'])

lob2 = df_Lob2.resample('1D', on='DateTime').last()

lob2.dropna(subset=['highest_bid_price'], inplace=True)

dataframe2 = pd.concat([prices2,lob2],axis=1)
dataframe2 = dataframe2.drop(columns=['DateTime'])

dataframe2.index.name = 'DateTime'
dataframe2.head()

,Low,High,Open,Close,Volume,highest_bid_price,highest_bid_quantity,lowest_ask_price,lowest_ask_quantity,Spread,Mid_price,Market_depth
DateTime,,,,,,,,,,,,
2022-01-04,229.0,275.0,253.0,241.0,63484.0,240.0,7.0,241.0,1.0,1.0,240.5,773.0
2022-01-05,229.0,260.0,251.0,246.0,63584.0,243.0,10.0,244.0,1.0,1.0,243.5,752.0
2022-01-06,236.0,307.0,242.0,283.0,61091.0,282.0,11.0,283.0,1.0,1.0,282.5,734.0
2022-01-07,257.0,310.0,286.0,263.0,59114.0,262.0,11.0,263.0,1.0,1.0,262.5,766.0
2022-01-10,238.0,278.0,262.0,242.0,59570.0,238.0,5.0,239.0,1.0,1.0,238.5,697.0


In [33]:
# Test set B03
#Drop columns that are not needed
df3 = data3.drop(columns=['Party1','Party2','Bid','Ask'])

# new data frame with split value columns
new3 = df3["Date"].str.split("-", n = 2, expand = True)
  
# making separate first name column from new data frame
df3['Y'] = new3[0]
# making separate last name column from new data frame
df3['M'] = new3[1]
df3['D'] = new3[2]

df3[["Y", "M","D"]] = df3[["Y", "M","D"]].apply(pd.to_numeric)

#Create datetime column based on  given Time column
dt3 = []
for i in range(len(df3)):
    start3 = datetime(df3['Y'][i], df3['M'][i], df3['D'][i], 8, 0, 0) #start time when market opens
    dt3.append(start3 + timedelta(seconds = df3["Time"][i]))

df3['DateTime'] = dt3

#High, low, open and close prices for every 1 Day interval
prices3 = df3.resample('1D', on='DateTime').Price.agg(['min','max', 'first','last'])
prices3.reset_index(inplace=True)

df_Q3 = df3.drop(columns=['Date', 'Time', 'Price', 'Y', 'M', 'D','highest_bid_price','highest_bid_quantity','lowest_ask_price','lowest_ask_quantity','Spread','Mid_price','Market_depth'])

df_Q3[["Quantity"]] = df3[["Quantity"]].apply(pd.to_numeric)

qty3 = df_Q3.resample('1D', on='DateTime').sum()

Quantity3 = qty3['Quantity'].tolist()

prices3['Quantity'] = Quantity3
prices3.dropna(subset=['min'], inplace=True)

prices3.rename(columns = {'DateTime':'Date', 'min':'Low', 'max':'High', 'first':'Open', 'last':'Close', 'Quantity':'Volume'}, inplace = True)
prices3.set_index('Date', inplace=True)

df_Lob3 = df3.drop(columns=['Quantity','Date', 'Time', 'Price', 'Y', 'M', 'D'])

lob3 = df_Lob3.resample('1D', on='DateTime').last()

lob3.dropna(subset=['highest_bid_price'], inplace=True)

dataframe3 = pd.concat([prices3,lob3],axis=1)
dataframe3 = dataframe3.drop(columns=['DateTime'])

dataframe3.index.name = 'DateTime'
dataframe3.head()

,Low,High,Open,Close,Volume,highest_bid_price,highest_bid_quantity,lowest_ask_price,lowest_ask_quantity,Spread,Mid_price,Market_depth
DateTime,,,,,,,,,,,,
2022-01-04,203.0,268.0,263.0,214.0,67870.0,213.0,9.0,214.0,1.0,1.0,213.5,763.0
2022-01-05,203.0,238.0,205.0,222.0,72179.0,221.0,9.0,222.0,1.0,1.0,221.5,750.0
2022-01-06,216.0,279.0,221.0,268.0,64726.0,267.0,9.0,268.0,1.0,1.0,267.5,714.0
2022-01-07,232.0,277.0,277.0,241.0,64277.0,240.0,7.0,241.0,1.0,1.0,240.5,781.0
2022-01-10,216.0,254.0,242.0,235.0,65274.0,234.0,6.0,235.0,1.0,1.0,234.5,777.0


In [34]:
dataframe.to_csv(f"dataframe1.csv",index = False)

In [35]:
dataframe2.to_csv(f"dataframe2.csv",index = False)
dataframe3.to_csv(f"dataframe3.csv",index = False)

In [36]:
prices.to_csv(f"prices_ohlc.csv",index = False)